In [ ]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])

In [ ]:
train_data = torchvision.datasets.MNIST(root = '/content', train = True, transform = transform, download = True)
test_data = torchvision.datasets.MNIST(root = '/content', train = False, transform = transform, download = True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = 128, shuffle = True, num_workers = 2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = 128, shuffle = True, num_workers = 2)

In [ ]:
image, label = train_data[0]
image.size()

torch.Size([1, 28, 28])

In [ ]:
classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
class NeuralNet(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 12, kernel_size = 3) # (28 - 3)/1 + 1 = 26. Output: (12, 26, 26)
    self.pool = nn.MaxPool2d(2, 2)  # (12, 13, 13)
    self.conv2 = nn.Conv2d(in_channels = 12, out_channels = 24, kernel_size = 4)  # (13-4) / 1 + 1 = 10. Output: (24, 10, 10)

    self.fc1 = nn.Linear(24*5*5, 256)
    self.fc2 = nn.Linear(256, 64)
    self.fc3 = nn.Linear(64, 10)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.pool(x)

    x = self.conv2(x)
    x = F.relu(x)
    x = self.pool(x)

    x = torch.flatten(x, start_dim = 1)

    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)

    return x

In [ ]:
net = NeuralNet()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 1e-3)

In [ ]:
for epoch in range(12):
  print(f'Training epoch {epoch}')
  running_loss = 0

  for i, data in enumerate(train_loader):
    inputs, labels = data

    optimizer.zero_grad()

    outputs = net(inputs)

    loss = loss_fn(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  print(f"Loss: {running_loss/len(train_loader):.4f}")

Training epoch 0
Loss: 0.2983
Training epoch 1
Loss: 0.0692
Training epoch 2
Loss: 0.0486
Training epoch 3
Loss: 0.0369
Training epoch 4
Loss: 0.0296
Training epoch 5
Loss: 0.0238
Training epoch 6
Loss: 0.0206
Training epoch 7
Loss: 0.0185
Training epoch 8
Loss: 0.0146
Training epoch 9
Loss: 0.0122
Training epoch 10
Loss: 0.0101
Training epoch 11
Loss: 0.0089


In [ ]:
torch.save(net.state_dict(), 'trained_net.pth')

In [ ]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

<All keys matched successfully>

In [ ]:
correct = 0
total = 0

net.eval()
with torch.no_grad():
  for data in test_loader:
    images, labels = data
    outputs = net(images)
    _, prediction = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (prediction == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy}')

Accuracy: 99.11


In [ ]:
import pandas as pd
def generate_kaggle_submission(model, output_csv_path="submission.csv",
    batch_size=128, device=None):

    model.eval()

    test_dataset = torchvision.datasets.MNIST(
    root="./data",
    train=False,
    download=True,
    transform=transform
)


    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False
    )

    all_predictions = []

    with torch.no_grad():
        for images, _ in test_loader:

            outputs = net(images)
            _, preds = torch.max(outputs, 1)
            all_predictions.append(preds)

    all_predictions = torch.cat(all_predictions).numpy()

    submission_df = pd.DataFrame({
        "ImageId": range(1, len(all_predictions) + 1),
        "Label": all_predictions
    })

    submission_df.to_csv(output_csv_path, index=False)
    print(f"Submission file saved to {output_csv_path}")

generate_kaggle_submission(net)

Submission file saved to submission.csv
